In [7]:
import pandas as pd

In [8]:
d20_21 = pd.read_excel('DATOS HISTÓRICOS 2020_2021_TODAS ESTACIONES.xlsx', sheet_name='SURESTE3')
d22_23 = pd.read_excel('DATOS HISTÓRICOS 2022_2023_TODAS ESTACIONES.xlsx', sheet_name='SURESTE3')
df = pd.concat([d20_21,d22_23])

In [9]:
df['date'] = pd.to_datetime(df['date'])
df["date"] = (df["date"] - df.iloc[0,0]).apply(lambda x: x.value/3.6e+12)
df.head()

,date,CO,NO,NO2,NOX,O3,PM10,PM2.5,PRS,RAINF,RH,SO2,SR,TOUT,WSR,WDR
0,0.0,NaN,NaN,NaN,NaN,NaN,37.0,18.0,NaN,NaN,NaN,NaN,0.000,NaN,NaN,NaN
1,1.0,1.88,2.0,5.2,7.2,25.0,37.0,28.0,734.2,0.01,93.0,NaN,0.037,11.68,1.7,98.0
2,2.0,1.86,2.6,5.0,7.6,22.0,55.0,28.0,733.8,0.02,93.0,NaN,0.037,11.68,1.6,133.0
3,3.0,1.78,3.1,5.9,8.9,21.0,35.0,11.0,733.0,0.00,93.0,NaN,0.037,11.70,1.5,198.0
4,4.0,1.76,2.0,4.5,6.5,22.0,27.0,NaN,732.8,0.02,93.0,NaN,0.037,11.80,0.7,240.0


In [10]:
from sklearn.impute import KNNImputer as imputer
k = imputer(n_neighbors=4)
datos = k.fit_transform((df.iloc[:, :]).to_numpy())

In [11]:
limpios = pd.DataFrame(datos, columns=df.columns)
limpios

,date,CO,NO,NO2,NOX,O3,PM10,PM2.5,PRS,RAINF,RH,SO2,SR,TOUT,WSR,WDR
0,0.0,1.8775,4.3,6.375,10.625,19.25,37.0,18.00,733.025,0.015,92.75,0.850,0.000,12.2675,1.325,207.25
1,1.0,1.8800,2.0,5.200,7.200,25.00,37.0,28.00,734.200,0.010,93.00,1.775,0.037,11.6800,1.700,98.00
2,2.0,1.8600,2.6,5.000,7.600,22.00,55.0,28.00,733.800,0.020,93.00,0.750,0.037,11.6800,1.600,133.00
3,3.0,1.7800,3.1,5.900,8.900,21.00,35.0,11.00,733.000,0.000,93.00,0.725,0.037,11.7000,1.500,198.00
4,4.0,1.7600,2.0,4.500,6.500,22.00,27.0,10.25,732.800,0.020,93.00,0.775,0.037,11.8000,0.700,240.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31785,31795.0,0.7600,3.2,6.900,9.900,36.00,107.0,15.00,727.800,0.000,19.00,3.600,0.079,38.1800,12.700,54.00
31786,31796.0,0.7100,2.6,11.300,13.700,29.00,60.0,9.00,728.300,0.000,23.00,3.600,0.003,36.3800,8.900,50.00
31787,31797.0,0.7200,2.9,10.800,13.500,26.00,74.0,3.00,728.700,0.000,26.00,3.700,0.001,35.1000,10.500,56.00
31788,31798.0,0.6800,3.0,10.100,12.800,25.00,112.0,2.00,729.400,0.000,40.00,15.300,0.001,33.6000,16.900,86.00


In [12]:
intervalos={
    "SO2":[0.008, 0.11, 0.165, 0.22],
    "NO2":[8.75, 11, 13.3, 15.5],
    "etiqueta": ["bueno", "aceptable", "mala", "muy mala"]
}
def change(val, col):
    if val< intervalos[col][0]:
        return "bueno"
    if val< intervalos[col][1]:
        return "aceptable"
    if val< intervalos[col][2]:
        return "mala"
    if val< intervalos[col][3]:
        return "muy mala"
    return "Extremadamente mala"

In [13]:
limpios["NO2_discret"] = limpios["NO2"].apply(lambda x: change(x, "NO2"))
limpios["SO2_discret"] = (limpios["SO2"]*0.001).apply(lambda x: change(x, "SO2"))

In [14]:
limpios["SO2_pm"] = limpios["SO2"]*0.001

In [15]:
limpios.head()

,date,CO,NO,NO2,NOX,O3,PM10,PM2.5,PRS,RAINF,RH,SO2,SR,TOUT,WSR,WDR,NO2_discret,SO2_discret,SO2_pm
0,0.0,1.8775,4.3,6.375,10.625,19.25,37.0,18.00,733.025,0.015,92.75,0.850,0.000,12.2675,1.325,207.25,bueno,bueno,0.000850
1,1.0,1.8800,2.0,5.200,7.200,25.00,37.0,28.00,734.200,0.010,93.00,1.775,0.037,11.6800,1.700,98.00,bueno,bueno,0.001775
2,2.0,1.8600,2.6,5.000,7.600,22.00,55.0,28.00,733.800,0.020,93.00,0.750,0.037,11.6800,1.600,133.00,bueno,bueno,0.000750
3,3.0,1.7800,3.1,5.900,8.900,21.00,35.0,11.00,733.000,0.000,93.00,0.725,0.037,11.7000,1.500,198.00,bueno,bueno,0.000725
4,4.0,1.7600,2.0,4.500,6.500,22.00,27.0,10.25,732.800,0.020,93.00,0.775,0.037,11.8000,0.700,240.00,bueno,bueno,0.000775


In [17]:
datos = limpios[["date","NO","NO2", "NOX", "SO2", "TOUT", "RH", "SR", "NO2_discret",	"SO2_discret",	"SO2_pm"]]

In [18]:
datos.head()

,date,NO,NO2,NOX,SO2,TOUT,RH,SR,NO2_discret,SO2_discret,SO2_pm
0,0.0,4.3,6.375,10.625,0.850,12.2675,92.75,0.000,bueno,bueno,0.000850
1,1.0,2.0,5.200,7.200,1.775,11.6800,93.00,0.037,bueno,bueno,0.001775
2,2.0,2.6,5.000,7.600,0.750,11.6800,93.00,0.037,bueno,bueno,0.000750
3,3.0,3.1,5.900,8.900,0.725,11.7000,93.00,0.037,bueno,bueno,0.000725
4,4.0,2.0,4.500,6.500,0.775,11.8000,93.00,0.037,bueno,bueno,0.000775


In [19]:
datos.to_csv("limpios.csv")